In [1]:
#import necessary libraries
import pandas as pd
import csv
import re
import validators
import emoji
import unidecode
import nltk
import pickle
nltk.download('stopwords')
import numpy as np
from sklearn.model_selection import train_test_split
from nltk.tokenize import WhitespaceTokenizer
from itertools import chain
import time
from sklearn.feature_extraction.text import TfidfVectorizer
import keras
from keras.preprocessing import sequence
import tensorflow as tf
from keras import Input
from keras import optimizers
from keras import backend as K
import torch
from keras import regularizers
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
from sklearn.datasets import make_blobs
from keras.models import Sequential
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from keras.layers import Dense, Activation, Dropout, Flatten
from keras.layers import Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D 
from keras.utils import plot_model
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from matplotlib import pyplot
from keras.models import load_model
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_selection import SelectPercentile, f_classif

import tensorflow as tf
from tensorflow import keras
from keras import optimizers
from keras import regularizers
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten
from keras.layers import Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D 
from keras.utils import np_utils
from keras.utils import plot_model
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from itertools import chain

from tqdm import tqdm
from gensim.models import fasttext
from gensim.test.utils import datapath
import os, re, csv, math, codecs, pickle, nltk
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

[nltk_data] Downloading package stopwords to C:\Users\Kirby
[nltk_data]     Wenceslao\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
#Load test and testresutls   

with open('x_test.pkl', 'rb') as file:
    x_test = pickle.load(file)
    
with open('y_test.pkl', 'rb') as file:
    y_test = pickle.load(file)
    
with open('fastTextCNN_test_results.pkl', 'rb') as file:
    fastTextCNN_test_results = pickle.load(file) 
    
with open('TFIDFFFNN_test_results.pkl', 'rb') as file:
    TFIDFFFNN_test_results = pickle.load(file)  
    
#Convert Labels from Strings to categorical Integers {Non-Hate = 1, Hate = 0}


df_ytest = pd.DataFrame(y_test, columns = ['Label'])

fastTextCNN_test_results.columns = ['Tokens', 'Label', 'Prediction']
fastTextCNN_test_results['Label'] = fastTextCNN_test_results['Label'].str.get(0)
fastTextCNN_test_results['Prediction'] = fastTextCNN_test_results['Prediction'].str.get(0)

fastTextCNN_tweets_joined = []

for i in range(len(fastTextCNN_test_results.Tokens.values)):
    tweet = " ".join(fastTextCNN_test_results.Tokens.values[i])
    fastTextCNN_tweets_joined.append(tweet)
    
fastTextCNN_test_results.insert(0, "Tweet", fastTextCNN_tweets_joined, True)


TFIDFFFNN_test_results.columns = ['Tokens', 'Label', 'Prediction']

tfidfffnn_tweets_joined = []
for i in range(len(TFIDFFFNN_test_results.Tokens.values)):
    tweet = " ".join(TFIDFFFNN_test_results.Tokens.values[i])
    tfidfffnn_tweets_joined.append(tweet)
    
TFIDFFFNN_test_results.insert(0, "Tweet", tfidfffnn_tweets_joined, True)
#TFIDFFFNN_test_results['Label'] = TFIDFFFNN_test_results['Label'].str.get(0)
#TFIDFFFNN_test_results['Prediction'] = TFIDFFFNN_test_results['Prediction'].str.get(0)
#flatten training set for fitting on tfidf
#x_train_fit = list(chain.from_iterable(x_train))

mapping = {'Non-hate': 0, 'Hate': 1}
mapping_reverse = {0: 'Non-hate', 1: 'Hate'}
fastTextCNN_test_results = fastTextCNN_test_results.replace({'Label': mapping_reverse})
TFIDFFFNN_test_results = TFIDFFFNN_test_results.replace({'Label': mapping_reverse})
df_ytest = df_ytest.replace({'Label': mapping})

In [3]:
#collect index of misclassified and remove from results


fasttext_misclassified = []
tfidf_misclassified = []

for i in range(len(y_test)):
    if(y_test[i] != fastTextCNN_test_results.Label.values[i]):
        fasttext_misclassified.append(i)
        
    if(y_test[i] != TFIDFFFNN_test_results.Label.values[i]):
        tfidf_misclassified.append(i)
    
print(len(fasttext_misclassified))
print(len(tfidf_misclassified))

#remove misclassified
fastTextCNN_test_results.drop(fasttext_misclassified, inplace = True )
TFIDFFFNN_test_results.drop(tfidf_misclassified, inplace = True )

print()

83
85



In [4]:
def dummy_fun(doc):
    return doc

def get_keywords(data,filename):
    print(filename)
    tfidf = TfidfVectorizer(
        analyzer='word',
        tokenizer=dummy_fun,
        preprocessor=dummy_fun,
        token_pattern=None)  

    #fit text to tfidf
    tfidf.fit_transform(data)


    fasttext_dict = list(zip(tfidf.get_feature_names(), tfidf.idf_))
    fasttext_df = pd.DataFrame(fasttext_dict, columns = ['Token', 'TFIDF_Value'])
    fasttextresults_label_sorted = fasttext_df.sort_values(by=['TFIDF_Value'], ascending=False)
    fasttextresults_label_sorted.to_csv(filename, index=False)
    
    return fasttextresults_label_sorted

In [5]:
TFIDFFFNN_test_results.to_csv('TFIDFFFNN_test_results.csv', index=False)
TFIDFFFNN_test_results

,Tweet,Tokens,Label,Prediction
0,day ends balota kiko pangilinan kasama,"[day, ends, balota, kiko, pangilinan, kasama]",Non-hate,0.999681
1,president cutie,"[president, cutie]",Non-hate,1.000000
2,first day covid pandemic via wow vp true leade...,"[first, day, covid, pandemic, via, wow, vp, tr...",Non-hate,0.999829
3,presidente bise presidente kiko pangilininannnn,"[presidente, bise, presidente, kiko, pangilini...",Non-hate,0.983980
4,dalawang pangakong binitiwan sakaling ibotong ...,"[dalawang, pangakong, binitiwan, sakaling, ibo...",Non-hate,0.999793
...,...,...,...,...
507,robredo's final speech comelec's presidential ...,"[robredo's, final, speech, comelec's, presiden...",Non-hate,0.991915
508,sinungaling umpisa sinungaling kahuli hulihan ...,"[sinungaling, umpisa, sinungaling, kahuli, hul...",Hate,1.000000
509,siniraan pilipinas last six years wants reward...,"[siniraan, pilipinas, last, six, years, wants,...",Hate,0.985040
510,manalo matalo kay hganggang dulo,"[manalo, matalo, kay, hganggang, dulo]",Non-hate,0.999960


In [6]:
groupedby_ffnn_label = TFIDFFFNN_test_results.groupby(TFIDFFFNN_test_results.Label)

#group by label
ffnn_nonhate = groupedby_ffnn_label.get_group("Non-hate")
ffnn_hate = groupedby_ffnn_label.get_group("Hate")

In [7]:
#tfidf highest keywords ffnn hate
get_keywords(ffnn_hate.Tokens.values,'TFIDFFFNN_hate_keywords.csv').head(20)

TFIDFFFNN_hate_keywords.csv


,Token,TFIDF_Value
0,aaral,5.68675
1064,nlng,5.68675
1061,niyan,5.68675
1060,ninyo,5.68675
1059,ninakaw,5.68675
1058,nina,5.68675
1057,niloko,5.68675
1056,nilangae,5.68675
1055,nila.sila,5.68675
1054,nikki,5.68675


In [8]:
#tfidf highest keywords ffnn nonhate
get_keywords(ffnn_nonhate.Tokens.values,'TFIDFFFNN_nonhate_keywords.csv').head(20)

TFIDFFFNN_nonhate_keywords.csv


,Token,TFIDF_Value
0,aadvance,5.663439
603,pamasahe,5.663439
592,pagsulong,5.663439
594,pagtitiis,5.663439
595,pagtratrabaho,5.663439
596,pahintuin,5.663439
597,pake,5.663439
598,pakikikipagkampanya,5.663439
599,pakikipagusap,5.663439
600,palampasin,5.663439


In [9]:
#hate
ffnn_hate

,Tweet,Tokens,Label,Prediction
6,pagsinabi sarah sasabihin niyo sarah geronimo ...,"[pagsinabi, sarah, sasabihin, niyo, sarah, ger...",Hate,0.999997
7,ferdinand jr.--the unapologetic corrupt dictat...,"[ferdinand, jr.--the, unapologetic, corrupt, d...",Hate,0.999953
8,really true millenials rooting since experienc...,"[really, true, millenials, rooting, since, exp...",Hate,0.997435
11,natatae ata dami kagad kuda kay edi binasag ka...,"[natatae, ata, dami, kagad, kuda, kay, edi, bi...",Hate,0.998242
13,nga saken worth si maam porket lutang ibang in...,"[nga, saken, worth, si, maam, porket, lutang, ...",Hate,0.999901
...,...,...,...,...
504,mirror image duterte si parehong palaaway bast...,"[mirror, image, duterte, si, parehong, palaawa...",Hate,0.999910
506,ilocana mas madami pang credentials kesa kay,"[ilocana, mas, madami, pang, credentials, kesa...",Hate,0.908417
508,sinungaling umpisa sinungaling kahuli hulihan ...,"[sinungaling, umpisa, sinungaling, kahuli, hul...",Hate,1.000000
509,siniraan pilipinas last six years wants reward...,"[siniraan, pilipinas, last, six, years, wants,...",Hate,0.985040


In [10]:
#top ffnn hate sentences sorted by prediction
ffnn_hate_sortedbyprediction = ffnn_hate.sort_values(by=['Prediction'], ascending=False)
ffnn_hate_sortedbyprediction.to_csv('TFIDFFFNN_hate_results.csv', index=False)
ffnn_hate_sortedbyprediction

,Tweet,Tokens,Label,Prediction
115,rin ayaw makabalik lalo si makikita lang pag e...,"[rin, ayaw, makabalik, lalo, si, makikita, lan...",Hate,1.000000
119,obvious namang binigay yung questions kay liar...,"[obvious, namang, binigay, yung, questions, ka...",Hate,1.000000
72,marcosjr duwag ayaw umattend debate kayang ipa...,"[marcosjr, duwag, ayaw, umattend, debate, kaya...",Hate,1.000000
461,magnanakaw,[magnanakaw],Hate,1.000000
65,marcosjr duwag ayaw umattend debate kayang ipa...,"[marcosjr, duwag, ayaw, umattend, debate, kaya...",Hate,1.000000
...,...,...,...,...
364,naneto pinagsasasabe,"[naneto, pinagsasasabe]",Hate,0.547677
328,,[],Hate,0.547677
270,redtagger,[redtagger],Hate,0.547677
413,niyo magpalit kulay parin mababago dependent,"[niyo, magpalit, kulay, parin, mababago, depen...",Hate,0.531275


In [11]:
#for nonhate
ffnn_nonhate

,Tweet,Tokens,Label,Prediction
0,day ends balota kiko pangilinan kasama,"[day, ends, balota, kiko, pangilinan, kasama]",Non-hate,0.999681
1,president cutie,"[president, cutie]",Non-hate,1.000000
2,first day covid pandemic via wow vp true leade...,"[first, day, covid, pandemic, via, wow, vp, tr...",Non-hate,0.999829
3,presidente bise presidente kiko pangilininannnn,"[presidente, bise, presidente, kiko, pangilini...",Non-hate,0.983980
4,dalawang pangakong binitiwan sakaling ibotong ...,"[dalawang, pangakong, binitiwan, sakaling, ibo...",Non-hate,0.999793
...,...,...,...,...
499,davao del sur governor cagas endorses presiden...,"[davao, del, sur, governor, cagas, endorses, p...",Non-hate,0.999429
503,great mother divine mother restless children e...,"[great, mother, divine, mother, restless, chil...",Non-hate,0.966261
505,jodi sta maria nag-house-to-house campaign kik...,"[jodi, sta, maria, nag-house-to-house, campaig...",Non-hate,1.000000
507,robredo's final speech comelec's presidential ...,"[robredo's, final, speech, comelec's, presiden...",Non-hate,0.991915


In [12]:
#top ffnn nonhate sentences sorted by prediction
ffnn_nonhate_sortedbyprediction = ffnn_nonhate.sort_values(by=['Prediction'], ascending=False)
ffnn_nonhate_sortedbyprediction.to_csv('TFIDFFFNN_nonhate_results.csv', index=False)
ffnn_nonhate_sortedbyprediction

,Tweet,Tokens,Label,Prediction
300,please watch let's attract manifest vice presi...,"[please, watch, let's, attract, manifest, vice...",Non-hate,1.000000
457,president cutie,"[president, cutie]",Non-hate,1.000000
243,pag-asa yung tumutulak mangarap dulo kadiliman...,"[pag-asa, yung, tumutulak, mangarap, dulo, kad...",Non-hate,1.000000
367,true leaders show darating po best job woman vp,"[true, leaders, show, darating, po, best, job,...",Non-hate,1.000000
292,madam president maria leonor gerona cutie true...,"[madam, president, maria, leonor, gerona, cuti...",Non-hate,1.000000
...,...,...,...,...
77,yes,[yes],Non-hate,0.622230
348,update closing statement smni's presidential d...,"[update, closing, statement, smni's, president...",Non-hate,0.608880
452,vp clearly ready th president philippines,"[vp, clearly, ready, th, president, philippines]",Non-hate,0.565044
414,inday sara duterte let us unite future,"[inday, sara, duterte, let, us, unite, future]",Non-hate,0.556530


In [13]:
#For fasttext results

In [14]:
fastTextCNN_test_results.to_csv('fastTextCNN_test_results.csv', index=False)
fastTextCNN_test_results

,Tweet,Tokens,Label,Prediction
0,day ends balota kiko pangilinan kasama,"[day, ends, balota, kiko, pangilinan, kasama]",Non-hate,0.016057
1,president cutie,"[president, cutie]",Non-hate,0.016355
2,first day covid pandemic via wow vp true leade...,"[first, day, covid, pandemic, via, wow, vp, tr...",Non-hate,0.099809
3,presidente bise presidente kiko pangilininannnn,"[presidente, bise, presidente, kiko, pangilini...",Non-hate,0.021594
4,dalawang pangakong binitiwan sakaling ibotong ...,"[dalawang, pangakong, binitiwan, sakaling, ibo...",Non-hate,0.158146
...,...,...,...,...
507,robredo's final speech comelec's presidential ...,"[robredo's, final, speech, comelec's, presiden...",Non-hate,0.171745
508,sinungaling umpisa sinungaling kahuli hulihan ...,"[sinungaling, umpisa, sinungaling, kahuli, hul...",Hate,0.943714
509,siniraan pilipinas last six years wants reward...,"[siniraan, pilipinas, last, six, years, wants,...",Hate,0.907931
510,manalo matalo kay hganggang dulo,"[manalo, matalo, kay, hganggang, dulo]",Non-hate,0.130683


In [15]:
groupedby_cnn_label = fastTextCNN_test_results.groupby(fastTextCNN_test_results.Label)
#group by label
cnn_nonhate = groupedby_cnn_label.get_group("Non-hate")
cnn_hate = groupedby_cnn_label.get_group("Hate")

In [16]:
#get top keywords by tfidf

In [17]:
#fasttextcnn highest keywords nonhate
get_keywords(cnn_nonhate.Tokens.values,'fastTextCNN_nonhate_keywords.csv').head(20)

fastTextCNN_nonhate_keywords.csv


,Token,TFIDF_Value
0,aadvance,5.639572
555,pagkakaisa,5.639572
558,pagpunta,5.639572
559,pagsulong,5.639572
561,pagtitiis,5.639572
562,pagtratrabaho,5.639572
563,pakikikipagkampanya,5.639572
564,pakikipagusap,5.639572
565,palampasin,5.639572
566,palayain,5.639572


In [18]:
#fasttextcnn highest keywords hate
get_keywords(cnn_hate.Tokens.values,'fastTextCNN_hate_keywords.csv').head(20)

fastTextCNN_hate_keywords.csv


,Token,TFIDF_Value
0,aaral,5.718499
1136,ndf,5.718499
1134,nawala,5.718499
1133,natulungan,5.718499
1132,natitira,5.718499
1131,national,5.718499
1130,nation's,5.718499
1129,nation,5.718499
1126,naten,5.718499
1125,natatanggap,5.718499


In [19]:
#top cnn hate sentences sorted by prediction
cnn_hate_sortedbyprediction = cnn_hate.sort_values(by=['Prediction'], ascending=False)
cnn_hate_sortedbyprediction.to_csv('fastTextCNN_hate_results.csv', index=False)
cnn_hate_sortedbyprediction

,Tweet,Tokens,Label,Prediction
411,wag mong gawing parang tanga lang..hindi duwag...,"[wag, mong, gawing, parang, tanga, lang..hindi...",Hate,0.999912
198,biggest flaws nating supporters eh yung pagigi...,"[biggest, flaws, nating, supporters, eh, yung,...",Hate,0.999888
195,bobo daw si bobo lagay yan pano lang yung di i...,"[bobo, daw, si, bobo, lagay, yan, pano, lang, ...",Hate,0.999879
137,asang asa pokwang mananalo wag masyadong umasa...,"[asang, asa, pokwang, mananalo, wag, masyadong...",Hate,0.999410
267,ulol yung nag sasabing i'm inc vote ulol nilok...,"[ulol, yung, nag, sasabing, i'm, inc, vote, ul...",Hate,0.999408
...,...,...,...,...
84,whaaaaaaat lenlen copies bonbon bano dark alte...,"[whaaaaaaat, lenlen, copies, bonbon, bano, dar...",Hate,0.562894
482,intl news reporting leading elections sooooo f...,"[intl, news, reporting, leading, elections, so...",Hate,0.550358
349,lutang npa,"[lutang, npa]",Hate,0.544337
269,nga exciting part pupuntang amerika magpa abor...,"[nga, exciting, part, pupuntang, amerika, magp...",Hate,0.526599


In [20]:
cnn_nonhate

,Tweet,Tokens,Label,Prediction
0,day ends balota kiko pangilinan kasama,"[day, ends, balota, kiko, pangilinan, kasama]",Non-hate,0.016057
1,president cutie,"[president, cutie]",Non-hate,0.016355
2,first day covid pandemic via wow vp true leade...,"[first, day, covid, pandemic, via, wow, vp, tr...",Non-hate,0.099809
3,presidente bise presidente kiko pangilininannnn,"[presidente, bise, presidente, kiko, pangilini...",Non-hate,0.021594
4,dalawang pangakong binitiwan sakaling ibotong ...,"[dalawang, pangakong, binitiwan, sakaling, ibo...",Non-hate,0.158146
...,...,...,...,...
499,davao del sur governor cagas endorses presiden...,"[davao, del, sur, governor, cagas, endorses, p...",Non-hate,0.159229
503,great mother divine mother restless children e...,"[great, mother, divine, mother, restless, chil...",Non-hate,0.300837
505,jodi sta maria nag-house-to-house campaign kik...,"[jodi, sta, maria, nag-house-to-house, campaig...",Non-hate,0.075493
507,robredo's final speech comelec's presidential ...,"[robredo's, final, speech, comelec's, presiden...",Non-hate,0.171745


In [21]:
#top cnn nonhate sentences sorted by prediction
cnn_nonhate_sortedbyprediction = cnn_nonhate.sort_values(by=['Prediction'], ascending=True)
cnn_nonhate_sortedbyprediction.to_csv('fastTextCNN_nonhate_results.csv', index=False)
cnn_nonhate_sortedbyprediction

,Tweet,Tokens,Label,Prediction
470,nakikiisa pagsuporta kay vp sen kiko pangilina...,"[nakikiisa, pagsuporta, kay, vp, sen, kiko, pa...",Non-hate,0.001549
162,best job woman vp,"[best, job, woman, vp]",Non-hate,0.001608
243,pag-asa yung tumutulak mangarap dulo kadiliman...,"[pag-asa, yung, tumutulak, mangarap, dulo, kad...",Non-hate,0.001744
309,si yel yarbz swith nakiki-isa pagsuporta kay v...,"[si, yel, yarbz, swith, nakiki-isa, pagsuporta...",Non-hate,0.002290
18,gobyernong tapat angat buhay si volleydeity bi...,"[gobyernong, tapat, angat, buhay, si, volleyde...",Non-hate,0.002733
...,...,...,...,...
359,sara duterte pula pilipinas,"[sara, duterte, pula, pilipinas]",Non-hate,0.454136
49,philippines,[philippines],Non-hate,0.468089
490,west philippine sea pa-reach-out reach-out eh ...,"[west, philippine, sea, pa-reach-out, reach-ou...",Non-hate,0.469710
456,,[],Non-hate,0.469718
